# RLlib Sample Application: trivial-v0

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to trains a policy with the `trivial-v0` environment:

  - <https://github.com/DerwenAI/gym_trivial> 

This example is designed to illustrate how to build a minimal `gym` environment which implements all of the features required. Effectively, it simulates the operation of an `AND` gate in TTL which is trivial.

---

First, make sure that Ray and RLlib are installed…

In [ ]:
!pip install ray[rllib]
!pip install ray[debug]
!pip install ray[tune]
!pip install pandas
!pip install requests
!pip install tensorflow

Then start Ray…

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo

ray.shutdown()
ray.init(ignore_reinit_error=True)

After a successful launch, the last log output line should read `View the Ray dashboard at localhost:8265`

Open <http://localhost:8265/> in another tab to view the Ray dashboard as the example runs.

---

Next we'll train an RLlib policy with the `trivial-v0` environment <https://github.com/DerwenAI/gym_trivial>

In [ ]:
!pip install git+git://github.com/DerwenAI/gym_trivial.git#egg=pkg&subdirectory=gym-trivial

In [ ]:
from gym_trivial.envs.trivial_env import Trivial
from ray.tune.registry import register_env

register_env("trivial-v0", lambda config: Trivial())

By default, training runs for only `1` iteration. Increase the `n_iter` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/triv` directory.

In [ ]:
SELECT_ENV = "trivial-v0"
N_ITER = 1

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

reward_history = []

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

for _ in range(N_ITER):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save("/tmp/ppo/triv")
    print(f"\n{file_name}")

In [ ]:
print(reward_history)

Do the episode rewards increase after multiple iterations?
That shows how the policy is improving.

Also, print out the policy and model to see the results of training in detail…

In [ ]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

This visualizes the "trivial" agent operating within the simulation: setting either of two inputs until both are set.

**(NB: the `rollout` CLI script will not work with custom env)**

In [ ]:
! rllib rollout \
    /tmp/ppo/triv/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"trivial-v0\"}" --run PPO \
    --steps 2000

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

---

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (copy/paste the URL it generates) to visualize key metrics from training with RLlib…

In [ ]:
!tensorboard --logdir=~/ray_results/